In [136]:
import pandas as pd
import numpy as np
from nltk.tokenize import regexp_tokenize
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from gensim.models import Word2Vec
import gensim.corpora as corpora
import gensim
from datetime import datetime, timedelta
import time
import pickle
import pprint
import pdb

In [2]:
import nltk
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\fergu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\fergu\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\fergu\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
wnl = WordNetLemmatizer()

def func_lemmatize(words):
    lemmatized = []
    for word, tag in pos_tag(words):
        wntag = tag[0].lower()
        wntag = wntag if wntag in ['a','r','n','v'] else None
        
        lemma = wnl.lemmatize(word,wntag) if wntag else word
        lemmatized.append(lemma)
    return lemmatized

stop = set(stopwords.words('english'))

pattern = r'(\w+)'

In [129]:
df= pd.read_pickle('p25million.pkl')

In [130]:
df.shape

(250000, 13)

In [156]:
def lda_compute(df,num_topics,passes=20):
    start = time.time()
    id2word = corpora.Dictionary(df['article_words'])
    
    articles = df['article_words']
    
    
    corpus = [id2word.doc2bow(article) for article in articles]
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,id2word=id2word,num_topics=num_topics,passes=passes)
    
#    print(lda_model.print_topics())
    print('-' * 10)
    doc_lda = lda_model[corpus]
    
    lda_lists = [lda_model[article] for article in corpus]
    print(f'lda_lists[-1]: {lda_lists[-1]}')
    lda_coeffs = [[pair[1] for pair in article] for article in lda_lists]
    print(f'lda_coeffs[-1]: {lda_coeffs[-1]}')
    lda_inds = [[pair[0] for pair in article] for article in lda_lists]
    print(f'lda_inds[-1]: {lda_inds[-1]}')
    
    rogues = [i for i in range(len(lda_inds)) if len(lda_inds[i])==0]
    
    lda_lists = [lda_lists[i] for i in range(len(lda_lists)) if i not in rogues]
    lda_coeffs = [lda_coeffs[i] for i in range(len(lda_coeffs)) if i not in rogues]
    lda_inds = [lda_inds[i] for i in range(len(lda_inds)) if i not in rogues]
    
    df.drop(rogues,inplace=True,axis=0)
    
    
#    df_rogue = df.loc[rogues,:]
#    return df_rogue 
    
###    lda_argmax = [np.argmax(tup) if len(tup) < 0 else None for tup in lda_coeffs]
#    lda_args = [np.arsgort(tup)[::-1] for tup in lda_coeffs]
    
    lda_argmax = [np.argmax(tup) for tup in lda_coeffs]
    
    
    print(f'lda_argmax[-1]: {lda_argmax[-1]}')
#    lda_inds = [pair[0] for pair in lda_tuples]
#    print(lda_inds[-1])
#    print('-' * 10)
#    print(lda_argmax[-1])
#    print('-' * 10)
###    lda_topics = [lda_inds[i][lda_argmax[i]] if lda_argmax[i] else None for i in range(len(lda_lists))]
    lda_topics = [lda_inds[i][lda_argmax[i]] for i in range(len(lda_lists))]
    
    sec_threshold = 0.5
    
#    pdb.set_trace()
    
#    secondary_topics = [[coeff for coeff in lda_coeffs[i] if 1 > (coeff / lda_coeffs[i][lda_argmax[i]]) >= sec_threshold] for i in range(len(lda_lists))]
        
    
#    secondary_topics = [[coeff for coeff in lda_coeffs[i] if 1 > (coeff / lda_coeffs[i][lda_argmax[i]]) >= sec_threshold] for i in range(len(lda_lists))]
    
    
#    secondary topics = [[lda_inds[j] for j in range(len(lda_inds[i])) if lda_coeffs[j] / lda_coeffs[i][lda_argmax[i]] >= sec_threshold] for i in range(len(lda_lists))]      
    
    secondary_topics = [[pair[0] for pair in lda_lists[i] if 1 > pair[1] / lda_coeffs[i][lda_argmax[i]] >= sec_threshold] for i in range(len(lda_lists))]
    
    df['lda_topic'] = lda_topics
    
    df['other_topics'] = secondary_topics   
    
    interval = round((time.time() - start)/60,2)
    print(f'That  took {interval} mins.')
    return df, lda_model, corpus#, doc_lda

In [162]:
dft = df.copy()
#dft = df[:10000]

In [163]:
len(dft)

250000

In [166]:
#16:53
dft, lda_model, corpus = lda_compute(dft,num_topics=180,passes=7)

----------
lda_lists[-1]: [(5, 0.017492807), (71, 0.043794263), (75, 0.03264609), (118, 0.10579967), (132, 0.011139468), (133, 0.7105508), (176, 0.012388571), (179, 0.01699847)]
lda_coeffs[-1]: [0.017492807, 0.043794263, 0.03264609, 0.10579967, 0.011139468, 0.7105508, 0.012388571, 0.01699847]
lda_inds[-1]: [5, 71, 75, 118, 132, 133, 176, 179]
lda_argmax[-1]: 5
That  took 136.66 mins.


In [158]:
dft.shape

(10000, 15)

In [159]:
dft

,Unnamed: 0,Unnamed: 0.1,date,year,month,day,author,title,article,url,section,publication,article_words,lda_topic,other_topics
0,1527764,1527764,2017-08-28,2017,8.0,28,NaN,RPT-COLUMN-China's metals trade fragments into...,(Repeats Aug. 25 column without change. The op...,https://www.reuters.com/article/china-metals-a...,Intel,Reuters,"[repeats, aug, column, without, change, opinio...",3,[]
1,168893,168893,2016-04-08 15:50:02,2016,4.0,8,David Roberts,New music for you: Edna Vazquez plays her song...,"Once a month (or so), I'm bringing you an epis...",https://www.vox.com/culture/2016/4/8/11391828/...,NaN,Vox,"[month, bring, episode, pickathon, pumphouse, ...",7,[14]
2,2001409,2001409,2016-09-09 00:35:26,2016,9.0,9,The Associated Press,One Wild Pitch Leads to Two Runs as Indians Be...,Sports Briefing The Indians scored two runs on...,http://www.nytimes.com/2016/09/09/sports/baseb...,sports,The New York Times,"[sports, briefing, indians, score, two, run, d...",9,[]
3,358141,358141,2018-03-01 09:09:08,2018,3.0,1,Ron Miller,Dropbox to add native G Suite integration in n...,It’s been an eventful week for Dropbox coming ...,https://techcrunch.com/2018/03/01/dropbox-to-a...,NaN,TechCrunch,"[eventful, week, dropbox, come, announcement, ...",4,[18]
4,307943,307943,2017-12-14 00:00:00,2017,12.0,14,Christopher Matthews,Walmart will start giving salary advances to 1...,Walmart announced Wednesday that it will start...,https://www.axios.com/walm-1513267182-9615b668...,NaN,Axios,"[walmart, announce, wednesday, start, allow, w...",4,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1466493,1466493,2018-01-18,2018,1.0,18,NaN,BRIEF-Workspace Group Says Lettings In Q3 Aver...,Jan 18 (Reuters) - Workspace Group Plc: * ‍LET...,https://www.reuters.com/article/brief-workspac...,Financials,Reuters,"[jan, reuters, workspace, group, plc, lettings...",15,[]
9996,822082,822082,2017-08-06 14:00:00,2017,8.0,6,Char Adams,Justin Bieber Is 'Doing Better Already' After ...,Justin Bieber is “doing better already” after ...,https://people.com/celebrity/justin-bieber-can...,celebrity,People,"[justin, bieber, well, already, cancel, remain...",7,[]
9997,1317831,1317831,2017-12-18,2017,12.0,18,NaN,France files complaint against Amazon for abus...,PARIS (Reuters) - The French government has fi...,https://www.reuters.com/article/us-amazon-com-...,Business News,Reuters,"[paris, reuters, french, government, file, com...",18,[]
9998,2754503,2755263,2020-03-15 00:00:00,2020,3.0,15,"Herbert Lash, Marc Jones",Markets crater as coronavirus fears overwhelm ...,NEW YORK/LONDON (Reuters) - Markets reeled on ...,https://www.reuters.com/article/global-markets...,Business News,Reuters,"[new, york, london, reuters, markets, reel, mo...",3,[]


In [105]:
df.shape

(249941, 14)

In [167]:
dft.to_csv('df1_0_250k_lda_full(180).csv')

In [175]:
dft[dft['lda_topic']==2]

,Unnamed: 0,Unnamed: 0.1,date,year,month,day,author,title,article,url,section,publication,article_words,lda_topic,other_topics
54,1608835,1608835,2016-10-08,2016,10.0,8,David Milliken,UK Chancellor Hammond names HSBC economist as ...,LONDON (Reuters) - British Chancellor Philip H...,http://www.reuters.com/article/us-britain-poli...,Business News,Reuters,"[london, reuters, british, chancellor, philip,...",2,[]
66,2301664,2314028,2019-09-25 00:00:00,2019,9.0,25,"Noor Zainab Hussain, Samantha Machado",Model-maker Hornby says Brexit may delay its t...,(Reuters) - Now even model trains face possibl...,https://www.reuters.com/article/hornby-outlook...,Business News,Reuters,"[reuters, even, model, train, face, possible, ...",2,[]
99,1124253,1124253,2017-09-26,2017,9.0,26,NaN,"UK opposition ""scenario planning"" for run on s...","BRIGHTON, England, Sept 26 (Reuters) - Britain...",https://www.reuters.com/article/britain-politi...,Bonds News,Reuters,"[brighton, england, sept, reuters, britain, op...",2,[15]
198,2278555,2290919,2019-09-09 00:00:00,2019,9.0,9,NaN,Irish and British PMs establish common Brexit ...,DUBLIN (Reuters) - British Prime Minister Bori...,https://www.reuters.com/article/us-britain-eu-...,World News,Reuters,"[dublin, reuters, british, prime, minister, bo...",2,[]
419,1097351,1097351,2016-02-04,2016,2.0,4,Jamie Condliffe,BBC: UN Panel Believes Julian Assange Was 'Unl...,It’s almost four years since Julian Assange to...,https://gizmodo.com/bbc-un-panel-believes-juli...,NaN,Gizmodo,"[almost, four, year, since, julian, assange, t...",2,"[123, 164]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249539,1418376,1418376,2018-06-18,2018,6.0,18,William James,"May's Brexit laws rejected by parliament, Wedn...",LONDON (Reuters) - Prime Minister Theresa May’...,https://www.reuters.com/article/uk-britain-eu-...,Brexit,Reuters,"[london, reuters, prime, minister, theresa, ma...",2,[]
249581,2364141,2376505,2019-10-22 00:00:00,2019,10.0,22,NaN,No-deal more likely if lawmakers reject Brexit...,LONDON (Reuters) - A failure to pass a timetab...,https://www.reuters.com/article/uk-britain-eu-...,Brexit,Reuters,"[london, reuters, failure, pass, timetable, ge...",2,[]
249656,2582735,2583495,2020-01-14 00:00:00,2020,1.0,14,Huw Jones,Global banks urge EU to 'improve' market acces...,LONDON (Reuters) - International banks have ca...,https://www.reuters.com/article/britain-eu-ban...,Business News,Reuters,"[london, reuters, international, bank, call, e...",2,"[4, 166]"
249822,2390890,2403254,2019-10-28 00:00:00,2019,10.0,28,NaN,PM Johnson will respond to EU extension offer ...,LONDON (Reuters) - Prime Minister Boris Johnso...,https://www.reuters.com/article/uk-britain-eu-...,Brexit,Reuters,"[london, reuters, prime, minister, boris, john...",2,[]


In [54]:
dft['lda_topic'].value_counts()

50

In [11]:
a = [1,2,3,3,1]
np.argmax(a)

2

In [10]:
dft[dft['lda_topic']==0]

,Unnamed: 0,Unnamed: 0.1,date,year,month,day,author,title,article,url,section,publication,article_words,lda_topic
30,2499922,2499709,2019-12-07 00:00:00,2019,12.0,7,NaN,How NBA star Klay Thompson spent his first big...,Klay Thompson remembers the number on his firs...,https://www.cnbc.com/2019/12/07/how-nba-star-k...,NaN,CNBC,"[klay, thompson, remember, number, first, nba,...",0
41,12143,12143,2018-12-21 00:00:00,2018,12.0,21,NaN,White Ref Forces Black High School Wrestler to...,This is ridiculous ... A black high school wre...,https://www.tmz.com/2018/12/21/andrew-johnson-...,NaN,TMZ,"[ridiculous, black, high, school, wrestler, te...",0
49,2508545,2508332,2019-12-10 00:00:00,2019,12.0,10,Lisa Rapaport,Many summer camps may not be prepared for kids...,(Reuters Health) - Although most summer camps ...,https://www.reuters.com/article/us-health-alle...,Health News,Reuters,"[reuters, health, although, summer, camp, welc...",0
90,150510,150510,2019-02-08 00:00:00,2019,2.0,8,Arjun Panchadar,EA's 'Fortnite' rival wins 10 million gamers i...,(Reuters) - A game developed by Electronics Ar...,https://www.reuters.com/article/us-electronic-...,Business News,Reuters,"[reuters, game, develop, electronics, arts, in...",0
421,1330544,1330544,2016-04-07,2016,4.0,7,WIRED Staff,Great Meditation Apps That'll Help You Keep It...,"Taxes are due, students are prepping for final...",https://www.wired.com/2016/04/meditation-apps-...,gear,Wired,"[taxes, due, student, prepping, final, sunny, ...",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19279,402466,402466,2017-03-06 20:30:00,2017,3.0,6,Suzannah Weiss,"Transgender Bathroom Rights, Transphobic Teach...","Ishy Valdez, a high school student in Fort Wor...",https://www.refinery29.com/en-us/2017/03/14401...,NaN,Refinery 29,"[ishy, valdez, high, school, student, fort, wo...",0
19381,1487316,1487316,2017-08-24,2017,8.0,24,NaN,The winner of the $758.7 million Powerball jac...,"When you win the lottery, you suddenly have a ...",https://www.cnbc.com/2017/08/24/the-winner-of-...,NaN,CNBC,"[win, lottery, suddenly, lot, lose, financial,...",0
19986,164529,164529,2019-07-02 00:00:00,2019,7.0,2,NaN,Late bloomer Gatto-Monticone faces Serena in d...,LONDON (Reuters) - As Italian Giulia Gatto-Mon...,https://www.reuters.com/article/us-tennis-wimb...,Sports News,Reuters,"[london, reuters, italian, giulia, gatto, mont...",0
19994,634071,634071,2016-09-28 15:00:00,2016,9.0,28,By Lindsay Kimble,WATCH: The Friendship Collegiate Academy Footb...,The Friendship Collegiate Academy football tea...,https://people.com/sports/friendship-collegiat...,sports,People,"[friendship, collegiate, academy, football, te...",0


In [11]:
dft[dft['lda_topic']==1]

,Unnamed: 0,Unnamed: 0.1,date,year,month,day,author,title,article,url,section,publication,article_words,lda_topic
62,567267,567267,2019-04-27 00:00:00,2019,4.0,27,Cameron Faulkner,"Amazon Fire TV Edition 4K TVs start at $199, a...",Only the best deals on Verge-approved gadgets ...,https://www.theverge.com/good-deals/2019/4/27/...,Good Deals,The Verge,"[best, deal, verge, approve, gadget, get, good...",1
63,677617,677617,2018-06-06 00:00:00,2018,6.0,6,Leah Stodart,Walmart has KitchenAid stand mixers on sale fo...,"KitchenAid mixers are pretty damn nice, right?...",https://mashable.com/2018/06/06/kitchen-aid-mi...,NaN,Mashable,"[kitchenaid, mixer, pretty, damn, nice, right,...",1
67,525862,525862,2016-08-31 00:00:00,2016,8.0,31,Vlad Savov,Acer's Predator 21 X puts a curved screen and ...,Acer’s new Predator 21 X is a monster. Not onl...,https://www.theverge.com/circuitbreaker/2016/8...,Microsoft,The Verge,"[acer, new, predator, x, monster, machine, des...",1
105,2307720,2320084,2019-09-02 00:00:00,2019,9.0,2,Jon Porter,IFA 2019: the top announcements to expect from...,While Barcelona’s Mobile World Congress holds ...,https://www.theverge.com/2019/9/2/20840570/ifa...,Mobile,The Verge,"[barcelona, mobile, world, congress, hold, cro...",1
132,1036954,1036954,2016-07-12 00:00:00,2016,7.0,12,Raymond Wong,Amazon Prime Day: The best TV deals,Most recent update: 12:44 p.m. ET One of the b...,https://mashable.com/2016/07/12/best-amazon-pr...,NaN,Mashable,"[recent, update, p, et, one, biggest, shopping...",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19737,1123502,1123502,2018-04-10,2018,4.0,10,Andrew Liszewski,Garmin Made a Great Smartwatch for People Who ...,Photo: Andrew Liszewski (Gizmodo)There’s no b...,https://gizmodo.com/garmin-made-a-great-smartw...,Reviews,Gizmodo,"[photo, andrew, liszewski, gizmodo, better, ex...",1
19815,1032827,1032827,2019-04-02 00:00:00,2019,4.0,2,Chi Kwen Ngai,"iPad Pro, Echo Dot, Dell monitors, PNY microSD...","If you are new to the smart home world, there'...",https://mashable.com/shopping/best-deals-april...,NaN,Mashable,"[new, smart, home, world, nothing, well, start...",1
19848,1039376,1039376,2018-04-25 00:00:00,2018,4.0,25,Carlos Cadorniga,Apple's iPad mini 4 is on sale and would make ...,We loved the iPad mini 4 when it came out in 2...,https://mashable.com/2018/04/25/apple-ipad-min...,NaN,Mashable,"[love, ipad, mini, come, raymond, wong, say, a...",1
19933,445786,445786,2016-05-27 07:14:44,2016,5.0,27,Brian Heater,HP and MSI hope to convince you to carry a VR ...,A number of companies seem to have a firm gras...,https://techcrunch.com/2016/05/27/hp-msi-vr/,NaN,TechCrunch,"[number, company, seem, firm, grasp, headset, ...",1


In [12]:
dft[dft['lda_topic']==2]

,Unnamed: 0,Unnamed: 0.1,date,year,month,day,author,title,article,url,section,publication,article_words,lda_topic
910,127663,127663,2019-03-13 22:00:00,2019,3.0,13,Annabel Gat,"Daily Horoscopes: March 14, 2019",Can't get enough of your Broadly horoscopes? ...,https://www.vice.com/en_us/article/9kpmb3/dail...,Astro Guide,Vice,"[get, enough, broadly, horoscopes, join, us, c...",2
931,316836,316836,2017-03-17 07:00:00,2017,3.0,17,Annabel Gat,"Daily Horoscope: March 17, 2017","The Sun, in Pisces, challenges Saturn, in Sagi...",https://www.vice.com/en_us/article/nej45z/dail...,Astro Guide,Vice,"[sun, pisces, challenge, saturn, sagittarius, ...",2
1448,2686498,2687258,2020-02-03 04:00:07,2020,2.0,3,Jeremy Engle,Lesson of the Day: ‘These Images Show the Sun’...,"current events In this lesson, students will e...",https://www.nytimes.com/2020/02/03/learning/le...,learning,The New York Times,"[current, event, lesson, student, examine, sta...",2
1549,787422,787422,2017-06-27 00:00:00,2017,6.0,27,Yi Shu Ng,These mysterious light pillars surprised Malay...,This brilliant display of lights looks like it...,https://mashable.com/2017/06/27/light-pillars-...,NaN,Mashable,"[brilliant, display, light, look, like, come, ...",2
3289,1967354,1967354,2016-03-02 17:04:41,2016,3.0,2,Nicholas St. Fleur,Stealing a Clear Look at an Elusive Star,Trilobites Gleaming through these crimson gas ...,http://www.nytimes.com/2016/03/03/science/tens...,science,The New York Times,"[trilobites, gleaming, crimson, gas, cloud, ra...",2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19713,437277,437277,2017-01-13 08:59:33,2017,1.0,13,Emily Calandrelli,"Moon Express raises $20M in Series B-1, fully ...","Moon Express, a company competing in the Googl...",https://techcrunch.com/2017/01/13/moon-express...,NaN,TechCrunch,"[moon, express, company, compete, google, luna...",2
19793,544592,544592,2017-09-15,2017,9.0,15,Thuy Ong,Watch NASA’s Cassini mission come to an end 20...,NASA’s Cassini spacecraft will bid farewell la...,https://www.theverge.com/2017/9/15/16311502/wa...,Science,The Verge,"[nasa, cassini, spacecraft, bid, farewell, lat...",2
19813,1101205,1101205,2016-04-19,2016,4.0,19,Maddie Stone,We Just Learned Something Crazy About the Atmo...,Artist’s concept of Venus Express aerobraking....,https://gizmodo.com/we-just-learned-something-...,Space,Gizmodo,"[artist, concept, venus, express, aerobraking,...",2
19893,212262,212262,2016-04-26 15:00:00,2016,4.0,26,Alex Swerdloff,"35,000 People Gathered to Throw Dead Fish on a...","Southern culture is an enigmatic beast, with m...",https://www.vice.com/en_us/article/4x55zb/3500...,Food by VICE,Vice,"[southern, culture, enigmatic, beast, many, od...",2


In [13]:
dft[dft['lda_topic']==3]

,Unnamed: 0,Unnamed: 0.1,date,year,month,day,author,title,article,url,section,publication,article_words,lda_topic
1291,736798,736798,2017-04-04 00:00:00,2017,4.0,4,Kellen Beck,Blizzard Entertainment wins $8.7 million lawsu...,Blizzard Entertainment is delivering its messa...,https://mashable.com/2017/04/04/blizzard-cheat...,NaN,Mashable,"[blizzard, entertainment, deliver, message, lo...",3
2095,2040297,2040297,2017-05-21 00:05:11,2017,5.0,21,NaN,"Michelle Ker, Jacob Locke",Michelle Ker and Jacob Daniel Locke were marri...,https://www.nytimes.com/2017/05/21/fashion/wed...,fashion,The New York Times,"[michelle, ker, jacob, daniel, locke, marry, m...",3
2696,2435584,2435371,2019-07-19 03:24:00,2019,7.0,19,Kirsten Acuna,'The Lion King': Differences between remake an...,Warning: There are spoilers ahead for Disney's...,https://www.businessinsider.com/the-lion-king-...,NaN,Business Insider,"[warning, spoiler, ahead, disney, lion, king, ...",3
4970,1069541,1069541,2017-03-02,2017,3.0,2,NaN,EU mergers and takeovers (March 2),"BRUSSELS, March 2 (Reuters) - The following ar...",http://www.reuters.com/article/eu-mergers-idUS...,Market News,Reuters,"[brussels, march, reuters, following, merger, ...",3
5261,548954,548954,2018-03-02,2018,3.0,2,NaN,EU mergers and takeovers (March 2),"BRUSSELS, March 2 (Reuters) - The following ar...",https://www.reuters.com/article/eu-ma/eu-merge...,Market News,Reuters,"[brussels, march, reuters, following, merger, ...",3
6931,1114030,1114030,2017-05-04,2017,5.0,4,Hudson Hongo,Ghouls Celebrate Blood Feast,Your browser does not support HTML5 video tag....,https://gizmodo.com/ghouls-celebrate-blood-fea...,NaN,Gizmodo,"[browser, support, video, tag, click, view, or...",3
7341,979335,979335,2019-02-01,2019,2.0,1,NaN,EU mergers and takeovers (Feb 1),"BRUSSELS, Feb 1 (Reuters) - The following are ...",https://www.reuters.com/article/eu-ma/eu-merge...,Mergers & Acquisitions - Americas,Reuters,"[brussels, feb, reuters, following, merger, re...",3
9124,712247,712247,2016-12-27 13:56:00,2016,12.0,27,Dave Quinn,Stars React to the Death of Star Wars Icon Car...,Saddened costars and friends are mourning the ...,https://people.com/movies/stars-react-carrie-f...,movies,People,"[saddened, costars, friend, mourn, death, belo...",3
9938,1373326,1373326,2018-06-18,2018,6.0,18,NaN,EU mergers and takeovers (June 18),"BRUSSELS, June 18 (Reuters) - The following ar...",https://www.reuters.com/article/eu-ma/eu-merge...,Consumer Goods and Retail,Reuters,"[brussels, june, reuters, following, merger, r...",3
13451,608830,608830,2017-11-08 14:27:00,2017,11.0,8,David Canfield,"Jennifer Aniston, Reese Witherspoon TV Series ...",Jennifer Aniston and Reese Witherspoon are hea...,https://people.com/tv/jennifer-aniston-reese-w...,tv,People,"[jennifer, aniston, reese, witherspoon, head, ...",3


In [14]:
dft[dft['lda_topic']==4]

,Unnamed: 0,Unnamed: 0.1,date,year,month,day,author,title,article,url,section,publication,article_words,lda_topic
54,1608835,1608835,2016-10-08,2016,10.0,8,David Milliken,UK Chancellor Hammond names HSBC economist as ...,LONDON (Reuters) - British Chancellor Philip H...,http://www.reuters.com/article/us-britain-poli...,Business News,Reuters,"[london, reuters, british, chancellor, philip,...",4
66,2301664,2314028,2019-09-25 00:00:00,2019,9.0,25,"Noor Zainab Hussain, Samantha Machado",Model-maker Hornby says Brexit may delay its t...,(Reuters) - Now even model trains face possibl...,https://www.reuters.com/article/hornby-outlook...,Business News,Reuters,"[reuters, even, model, train, face, possible, ...",4
92,1069586,1069586,2018-10-16,2018,10.0,16,NaN,"Daimler, BMW offer concessions to ease EU conc...",BRUSSELS (Reuters) - German luxury carmakers D...,https://www.reuters.com/article/us-bmw-m-a-dai...,Business News,Reuters,"[brussels, reuters, german, luxury, carmaker, ...",4
99,1124253,1124253,2017-09-26,2017,9.0,26,NaN,"UK opposition ""scenario planning"" for run on s...","BRIGHTON, England, Sept 26 (Reuters) - Britain...",https://www.reuters.com/article/britain-politi...,Bonds News,Reuters,"[brighton, england, sept, reuters, britain, op...",4
189,1033688,1033688,2016-10-31 00:00:00,2016,10.0,31,Jan Strupczewski,Italy's response to Commission on 2017 budget ...,"BRUSSELS, Oct 31 (Reuters) - Italy’s response ...",http://www.reuters.com/article/eu-commission-i...,Bonds News,Reuters,"[brussels, oct, reuters, italy, response, euro...",4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19897,707081,707081,2017-02-27 00:00:00,2017,2.0,27,NaN,"Russian announces recall of 33,275 Nissan Qash...","MOSCOW, Feb 27 (Reuters) - Russia’s standards ...",http://www.reuters.com/article/russia-nissan-r...,Auto & Truck Manufacturers,Reuters,"[moscow, feb, reuters, russia, standard, agenc...",4
19949,2171733,2171733,2019-09-02 00:00:00,2019,9.0,2,NaN,"UPDATE 2-Georgian PM Bakhtadze steps down, war...","(Adds quote, detail) TBILISI, Sept 2 (Reuters)...",https://www.cnbc.com/2019/09/02/reuters-americ...,Wires,CNBC,"[adds, quote, detail, tbilisi, sept, reuters, ...",4
19957,2522652,2522439,2019-12-17 00:00:00,2019,12.0,17,NaN,UK PM Johnson: We should resist calls to break...,LONDON (Reuters) - British Prime Minister Bori...,https://www.reuters.com/article/us-britain-pol...,World News,Reuters,"[london, reuters, british, prime, minister, bo...",4
19967,2269805,2269805,2019-09-06 00:00:00,2019,9.0,6,NaN,Germany urges Iran to change course on nuclear...,BERLIN (Reuters) - Germany said on Friday it w...,https://www.reuters.com/article/us-iran-nuclea...,World News,Reuters,"[berlin, reuters, germany, say, friday, late, ...",4


In [15]:
dft[dft['lda_topic']==5]

,Unnamed: 0,Unnamed: 0.1,date,year,month,day,author,title,article,url,section,publication,article_words,lda_topic
15,2644809,2645569,2020-02-10 00:00:00,2020,2.0,10,Justine Coleman,Puerto Rico-born US citizen wins lawsuit again...,A Puerto Rico-born U.S. citizen has won a laws...,https://thehill.com/homenews/state-watch/48240...,NaN,The Hill,"[puerto, rico, bear, u, citizen, win, lawsuit,...",5
52,88351,88351,2016-02-15 00:00:00,2016,2.0,15,Melanie Gouby,Ugandan Opposition Rally Ends in Tear Gas and ...,It's a cat and mouse game they have played for...,https://news.vice.com/en_us/article/vb8pya/uga...,NaN,Vice News,"[cat, mouse, game, play, long, time, dr, kizza...",5
68,2006340,2006340,2016-10-21 18:34:24,2016,10.0,21,Elisabeth Malkin,Mexico Arrests Ex-Chief of Police in City Wher...,MEXICO CITY — Mexico’s federal police on Frida...,http://www.nytimes.com/2016/10/22/world/americ...,world,The New York Times,"[mexico, city, mexico, federal, police, friday...",5
69,724750,724750,2016-02-26 00:30:00,2016,2.0,26,Michael Zennie,"Kansas Mass Shooting: Gunman Kills 3, Wounds 1...",Four people are dead and 14 have been wounded ...,https://people.com/crime/kansas-mass-shooting-...,crime,People,"[four, people, dead, wound, mass, shooting, fa...",5
87,2312027,2324391,2019-10-03 00:00:00,2019,10.0,3,NaN,Oscar De La Hoya Accused of Mistreating Staffe...,Oscar De La Hoya is being sued by a famous ex-...,https://www.tmz.com/2019/10/03/oscar-de-la-hoy...,NaN,TMZ,"[oscar, de, la, hoya, sue, famous, ex, golden,...",5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19852,1937483,1937483,2018-12-07 00:00:00,2018,12.0,7,Stephanie McNeal,This Guy Responded To Cops Who Posted His Mugs...,A police department in Washington state made a...,https://www.buzzfeednews.com/article/stephanie...,NaN,Buzzfeed News,"[police, department, washington, state, make, ...",5
19890,870581,870581,2016-08-16 00:00:00,2016,8.0,16,"By Ray Sanchez, CNN",What it's like to be black in Milwaukee,"Milwaukee, Wisconsin (CNN)On one side of Milwa...",https://www.cnn.com/2016/08/16/us/milwaukee-sh...,us,CNN,"[milwaukee, wisconsin, cnn, one, side, milwauk...",5
19969,258815,258815,2017-07-14 00:00:00,2017,7.0,14,Ranga Sirilal,U.N. raps Sri Lanka's slow progress on tacklin...,COLOMBO (Reuters) - The United Nations critici...,https://www.reuters.com/article/us-sri-lanka-r...,World News,Reuters,"[colombo, reuters, united, nations, criticize,...",5
19981,1910162,1910162,2016-03-16,2016,3.0,16,Leticia Miranda,Girl Died In Detention Cell From Irregular Hea...,Secretary of the Justice and Public Safety Cab...,https://www.buzzfeednews.com/article/leticiami...,NaN,Buzzfeed News,"[secretary, justice, public, safety, cabinet, ...",5


In [16]:
dft[dft['lda_topic']==6]

,Unnamed: 0,Unnamed: 0.1,date,year,month,day,author,title,article,url,section,publication,article_words,lda_topic
78,2636460,2637220,2020-02-09 00:00:00,2020,2.0,9,NaN,China's Hubei province reports 91 new deaths f...,BEIJING (Reuters) - The number of deaths in Ch...,https://www.reuters.com/article/us-china-healt...,World News,Reuters,"[beijing, reuters, number, death, china, centr...",6
176,2720676,2721436,2020-03-11 00:00:00,2020,3.0,11,Natasha Piñon,"Greta Thunberg encourages digital strikes, cit...","Once again, she's listening to the science. A...",https://mashable.com/article/coronavirus-greta...,NaN,Mashable,"[listen, science, precaution, take, around, wo...",6
202,2689332,2690092,2020-02-24 17:55:13,2020,2.0,24,"David D. Kirkpatrick, Farnaz Fassihi, Mujib Ma...",‘Recipe for a Massive Viral Outbreak’: Iran Em...,"Long a regional crossroads, Iran is spreading ...",https://www.nytimes.com/2020/02/24/world/middl...,world,The New York Times,"[long, regional, crossroad, iran, spread, new,...",6
290,2522124,2521911,2019-12-18 00:00:00,2019,12.0,18,Peter Szekely,That puppy in the window could make you sick a...,"NEW YORK (Reuters) - Beware of dog, especially...",https://www.reuters.com/article/us-usa-health-...,Health News,Reuters,"[new, york, reuters, beware, dog, especially, ...",6
363,2731063,2731823,2020-03-13 00:00:00,2020,3.0,13,"Gregory Scruggs, Thomson Reuters Foundation",Coronavirus sparks U.S. West Coast calls to ha...,"SEATTLE, March 13 (Thomson Reuters Foundation)...",https://www.reuters.com/article/coronavirus-sp...,Healthcare,Reuters,"[seattle, march, thomson, reuters, foundation,...",6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19665,2708672,2709432,2020-03-03 00:00:00,2020,3.0,3,Helen Coster,"High-fives, hugs, kisses discouraged as U.S. c...",(Reuters) - As news broke of more cases of cor...,https://www.reuters.com/article/health-coronav...,Technology News,Reuters,"[reuters, news, broke, case, coronavirus, spre...",6
19700,1499089,1499089,2016-06-03,2016,6.0,3,NaN,"Golfer Day raises doubts over Rio Games, citin...",(Reuters) - World number one golfer Jason Day ...,http://www.reuters.com/article/us-olympics-gol...,Sports News,Reuters,"[reuters, world, number, one, golfer, jason, d...",6
19704,781057,781057,2017-01-02 00:00:00,2017,1.0,2,NaN,"Smog returns, but Beijing says skies are getti...",BEIJING (Reuters) - The Chinese capital was on...,http://www.reuters.com/article/us-china-pollut...,Environment,Reuters,"[beijing, reuters, chinese, capital, second, h...",6
19763,2745147,2745907,2020-03-16 00:00:00,2020,3.0,16,"Giuseppe Fonte, Crispian Balmer",Italy pumps cash into coronavirus-hit economy ...,ROME (Reuters) - Italy approved 25 billion eur...,https://www.reuters.com/article/us-health-coro...,World News,Reuters,"[rome, reuters, italy, approve, billion, euro,...",6


In [17]:
dft[dft['lda_topic']==7]

,Unnamed: 0,Unnamed: 0.1,date,year,month,day,author,title,article,url,section,publication,article_words,lda_topic
9,590783,590783,2019-06-07 10:00:00,2019,6.0,7,Julie Mazziotta,Is Climate Change Making This Allergy Season t...,From the record-breaking rainfall in Californi...,https://people.com/health/climate-change-aller...,health,People,"[record, break, rainfall, california, recent, ...",7
604,875617,875617,2016-09-20 00:00:00,2016,9.0,20,"By Allison Chinchar, CNN",Foliage forecast: Drought may mute fall colors,"(CNN)When you think of fall, the first thing ...",https://www.cnn.com/2016/09/20/us/weather-fall...,us,CNN,"[cnn, think, fall, first, thing, usually, come...",7
657,1286721,1286721,2017-10-11,2017,10.0,11,Adela Suliman,Bunch of old bananas or building materials of ...,LONDON (Thomson Reuters Foundation) - Potatoes...,https://www.reuters.com/article/britain-constr...,Intel,Reuters,"[london, thomson, reuters, foundation, potatoe...",7
771,225277,225277,2016-10-29 16:53:00,2016,10.0,29,Eric Werner,Ceviche de Aguja with Ginger and Mezcal Recipe,Servings: 4-6Prep time: 35 minutesTotal time: ...,https://www.vice.com/en_us/article/ype9qj/cevi...,Food by VICE,Vice,"[servings, time, minutestotal, time, hour, ing...",7
773,1941220,1941220,2019-08-22 00:00:00,2019,8.0,22,Dan Whitcomb,"Homes, college evacuated as wildfire erupts in...",(Reuters) - A fast-moving wildfire that broke ...,https://www.reuters.com/article/us-california-...,Environment,Reuters,"[reuters, fast, moving, wildfire, break, thurs...",7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19434,2397330,2409694,2019-11-04 00:00:00,2019,11.0,4,Audrey Gray,Global Warming Is Already Destroying New Engla...,To wake up in the Northeastern United States—a...,https://newrepublic.com/article/155586/global-...,NaN,New Republic,"[wake, northeastern, united, states, californi...",7
19650,2553149,2553909,2020-01-03 00:00:00,2020,1.0,3,A. Odysseus Patrick,Australia fires: 'Houses were exploding' in in...,The statewide Total Fire Ban will continue tom...,https://www.washingtonpost.com/world/asia_paci...,NaN,Washington Post,"[statewide, total, fire, ban, continue, tomorr...",7
19697,2513632,2513419,2019-12-18 22:26:00,2019,12.0,18,Aylin Woodward,'Snow squall' hits New York City and Connectic...,"The National Weather Service issued a ""snow-sq...",https://www.businessinsider.com/what-is-snow-s...,NaN,Business Insider,"[national, weather, service, issue, snow, squa...",7
19789,2578906,2579666,2020-01-18 00:00:00,2020,1.0,18,Ian Livingston,"Updated forecast: Rain exits, leaving icy spot...",The steady precipitation has exited our region...,https://www.washingtonpost.com/weather/2020/01...,NaN,Washington Post,"[steady, precipitation, exit, region, leave, t...",7


In [18]:
dft.to_csv('lda_20k_50ktopics.csv')

In [24]:
len(corpus)

1000

In [25]:
lda_model[corpus[10]]

[(2, 0.26703453), (6, 0.33518597), (11, 0.39573386)]

In [26]:
lda_model[corpus[50]]

[(7, 0.9932477)]

In [27]:
lda_model[corpus[119]]

[(4, 0.035524845), (7, 0.01966797), (14, 0.49063125), (15, 0.44296286)]

In [28]:
lda_model[corpus[500]]

[(18, 0.9960151)]

In [29]:
lda_model[corpus[501]]

[(9, 0.3149924), (10, 0.6630205)]

In [30]:
lda_model[corpus[7]]

[(14, 0.16382726), (18, 0.8318437)]

In [31]:
lda_model[corpus[508]]

[(2, 0.76530415), (7, 0.20082355), (15, 0.031642746)]

In [34]:
np.argmax([lda_model[corpus[pair[1]]] for pair in lda_model[corpus[508]]])

TypeError: list indices must be integers or slices, not numpy.float32

In [8]:
df = df.dropna(subset = ['article'])

In [9]:
df.reset_index(inplace=True,drop=True)

In [11]:
df.shape

(2584165, 12)

In [10]:
df1 = df[:250000]
df2 = df[250000:500000]
df3 = df[500000:750000]
df4 = df[750000:1000000]
df5 = df[1000000:1250000]
df6 = df[1250000:1500000]
df7 = df[1500000:1750000]
df8 = df[1750000:2000000]
df9 = df[2000000:2250000]
df10 = df[2250000:]

In [19]:
num = 250000
np.random.seed(4)
inds_to_use = np.random.choice(df.index,num,replace=False)
df2 = df.loc[inds_to_use,:]
df2.reset_index(drop=True,inplace=True)

In [25]:
df2.shape

(250000, 13)

In [11]:
def clean(df):
    start = time.time()
    df['article_words'] = [regexp_tokenize(article,pattern) for article in df['article']]
    df['article_words'] = [[word for word in article if word.isalpha()] for article in df['article_words']]
    df['article_words'] = [func_lemmatize(article) for article in df['article_words']]
    df['article_words'] = [[word.lower() for word in article if word.lower() not in stop] for article in df['article_words']]
    interval = round((time.time() - start)/60,2)
    print(f'That  took {interval} mins.')
    return df

In [ ]:
# Started at 13:59
# Estd finish time: 15:52
df1 = clean(df1)

<ipython-input-11-49b0afa834b9>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['article_words'] = [regexp_tokenize(article,pattern) for article in df['article']]
<ipython-input-11-49b0afa834b9>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['article_words'] = [[word for word in article if word.isalpha()] for article in df['article_words']]


In [ ]:
df1.to_pickle('df1_0_250k.pkl')
df1.to_csv('df1_0_250k.csv')

In [23]:
df2.to_pickle('p25million.pkl')
df2.to_csv('p25million.csv')

In [18]:
df2.head(1)

,Unnamed: 0,Unnamed: 0.1,date,year,month,day,author,title,article,url,section,publication,article_words
0,1527764,1527764,2017-08-28,2017,8.0,28,NaN,RPT-COLUMN-China's metals trade fragments into...,(Repeats Aug. 25 column without change. The op...,https://www.reuters.com/article/china-metals-a...,Intel,Reuters,"[repeats, aug, column, without, change, opinio..."


In [26]:
def lda_compute(df,num_topics):
    start = time.time()
    id2word = corpora.Dictionary(df['article_words'])
    
    texts = df['article_words']
    
    
    corpus = [id2word.doc2bow(text) for text in texts]
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,id2word=id2word,num_topics=num_topics)
    
    print(lda_model.print_topics())
    print('-' * 10)
    doc_lda = lda_model[corpus]
    interval = round((time.time() - start)/60,2)
    print(f'That  took {interval} mins.')

In [28]:
lda_compute(df2,20)

[(0, '0.028*"say" + 0.009*"people" + 0.007*"health" + 0.004*"coronavirus" + 0.004*"year" + 0.004*"one" + 0.004*"case" + 0.004*"new" + 0.004*"go" + 0.004*"get"'), (1, '0.007*"use" + 0.007*"like" + 0.006*"one" + 0.006*"people" + 0.006*"say" + 0.005*"make" + 0.005*"new" + 0.005*"work" + 0.005*"time" + 0.005*"get"'), (2, '0.012*"say" + 0.008*"school" + 0.007*"student" + 0.005*"year" + 0.005*"state" + 0.005*"hill" + 0.004*"city" + 0.004*"new" + 0.004*"water" + 0.004*"home"'), (3, '0.011*"say" + 0.010*"market" + 0.010*"bank" + 0.009*"rate" + 0.008*"trade" + 0.008*"year" + 0.008*"u" + 0.006*"economy" + 0.005*"percent" + 0.005*"economic"'), (4, '0.008*"like" + 0.007*"say" + 0.007*"one" + 0.007*"get" + 0.006*"make" + 0.006*"go" + 0.005*"year" + 0.005*"time" + 0.005*"new" + 0.004*"show"'), (5, '0.013*"company" + 0.012*"say" + 0.006*"new" + 0.005*"use" + 0.005*"make" + 0.005*"one" + 0.005*"would" + 0.004*"business" + 0.004*"like" + 0.004*"year"'), (6, '0.012*"year" + 0.012*"percent" + 0.010*"say"

In [28]:
df.dropna(subset=['article'],inplace=True)
df.reset_index(drop=True,inplace=True)

In [29]:
df.shape

(2584165, 12)

In [10]:
## tab = pd.DataFrame({'article':['If the good guys are smart, this is the moment they swoop in and arrest everyone.',
##                                'Did you make it to the Allen & Co conference in Sun Valley, Idaho this past week?',
##                               'A pandemic era box office bonanza for Marvel’s Black Widow put a spring in Disney’s step today, though it hasn’t quite translated to gains for cinemas;',
##                               'This latest update, along with projections of positive economic growth, was met with notable optimism from the White House and record highs on Wall Street.',
##                               'In midst of this uneven recovery, and perhaps somewhat counterintuitively, others have noticed another pattern that may further elevate unemployment rates in the months to come:',
##                               'people are leaving their jobs.']})

In [ ]:
#11:30
df['article_words'] = [regexp_tokenize(article,pattern) for article in df['article']]
df['article_words'] = [[word for word in article if word.isalpha()] for article in df['article_words']]
df['article_words'] = [func_lemmatize(article) for article in df['article_words']]
df['article_words'] = [[word.lower() for word in article if word.lower() not in stop] for article in df['article_words']]

In [ ]:
df.to_pickle('df_cleaned.pkl')
df.to_csv('df_cleaned.csv')

In [ ]:
def create_w2v(col1,col2,max_epochs,vec_size,window,min_count,resume=False):
    now  = datetime.now()
    start_time_str = now.strftime('%H:%M:%S')
    print('started at',start_time_str)
        
    start_time = time.time()
    word_list = [[words for words in line] for line in col1]
    unpacked_word_list = [word for line in word_list for word in line]
    
    filename = 'model.sav'
    
    if resume == False:
        model = Word2Vec(col1,vector_size=vec_size,window=window,min_count=min_count,workers=-1)
        model.train(col1,total_examples=len(col1),epochs=max_epochs)
        print('word2vec model has been trained.')
        pickle.dump(model,open(filename,'wb'))
    else:
        file = open(filename,'rb')
        model = pickle.load(file)
        
    averaged_vectors = [] if resume == False else pd.read_csv('intermediate_results.csv',index_col=0)
    
    if resume == True:
        averaged_vectors = [list(averaged_vectors.loc[val,:]) for val in averaged_vectors.index]
    
    j = start_index= len(averaged_vectors)
    if j != 0:
        print(f'{round(j/len(col1),3) * 100}% complete.')
    start_checkpoint_time = time.time()
    
    progress_indicators = []
    
    
    for row in range(j,len(col1)):
        list_of_words = col1[row]
        list_of_vectors = [model.wv[word] for word in list_of_words if unpacked_word_list.count(word) >= min_count]
        avg_vector = np.mean(list_of_vectors,axis=0) if list_of_vectors else np.zeros(vec_size)
        averaged_vectors.append(avg_vector)
#        print(avg_vector)
        
        j += 1
#        print(f'{j} done')
        if j % 1000 == 0:
            intermediate_results = pd.DataFrame(averaged_vectors)
            intermediate_results.to_csv('intermediate_results.csv')
            
        percent_complete = round(j/len(col1) * 100)
        if (percent_complete % 1 == 0) & (percent_complete not in progress_indicators):
            cum_time_raw = (time.time() - start_time)/60
            cum_time = round((time.time() - start_time)/60,1)
            print('{}% complete. Time elapsed: {} min'.format(percent_complete,cum_time))
            progress_indicators.append(percent_complete)
            
            remaining_time = cum_time_raw * (len(col1) - j) / (j - start_index)
            
            est_finish_time = datetime.now() + timedelta(minutes=remaining_time)
            est_finish_time = est_finish_time.strftime('%D:%H:%M:%S')
            print(f'Estimated finish time: {est_finish_time}')
            print('-' * 15)
            
    df_of_vectors = pd.DataFrame(averaged_vectors)
        
    intermediate_table = pd.concat([col2,df_of_vectors],axis=1)
        
    rows_to_remove = [ind for ind in intermediate_table.index if intermediate_table.iloc[ind,2] == 0]
        
    removed_comments = pd.DataFrame(col2[rows_to_remove])
    final_table = intermediate_table.drop(rows_to_remove,axis=0)
        
    interval = round((time.time() - start_time)/60,1)
    print('-' * 25)
    print(f'Finished. That took {interval} min.')
    now = datetime.now()
    end_time_str = now.strftime('%H:%M:%S')
    print(f'Actual finish time: {end_time_str}')
        
    return final_table, removed_comments

In [ ]:
final_table, removed_comments = create_w2v(tab['article_words'],tab['tweet'],5,100,5,5)

In [142]:
final_table

,tweet,0,1,2,3,4,5,6,7,8,...,10,11,12,13,14,15,16,17,18,19
0,!!! RT @mayasolovely: As a woman you shouldn't...,0.010742,-0.004391,0.015802,-0.003414,0.003390,0.003548,-0.003043,-0.002796,-0.001108,...,-0.000829,0.012718,0.006761,-0.006497,-0.012051,0.007794,-0.009507,0.006585,0.005737,-0.001666
1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,0.004841,0.004436,0.000914,-0.009751,0.005038,0.011150,0.011603,0.009655,-0.012505,...,0.004664,0.002429,0.014794,0.004940,0.001413,-0.004927,0.012201,0.005936,-0.018094,0.001071
2,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,0.008423,0.012927,0.011020,0.010654,0.000597,-0.006037,0.007699,0.020504,-0.008069,...,-0.005035,-0.017148,0.020670,0.006083,-0.009244,-0.002631,0.001638,-0.007432,-0.004601,-0.016549
3,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,0.018069,-0.003312,0.002020,-0.004119,0.017109,-0.017962,0.002340,0.022885,0.002644,...,0.005931,-0.010217,0.019906,-0.022391,0.001863,0.004285,0.007125,-0.005956,0.020242,-0.004850
4,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,-0.009344,0.005417,-0.008419,0.004743,0.009811,-0.017030,0.007335,0.005161,-0.005933,...,0.005729,-0.005238,-0.000171,-0.005781,-0.016183,-0.009473,0.009901,-0.018340,0.006494,-0.018538
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24778,you's a muthaf***in lie &#8220;@LifeAsKing: @2...,-0.003729,-0.017738,0.009045,-0.016504,-0.010244,-0.000089,0.005051,0.001840,0.020293,...,-0.001044,0.008122,0.019703,0.007250,-0.005047,0.009291,0.010963,-0.000282,-0.033845,-0.003717
24779,"you've gone and broke the wrong heart baby, an...",0.002734,-0.002392,0.011020,-0.018769,-0.008788,0.001414,0.003678,0.012647,0.002893,...,0.001392,-0.001854,-0.005640,-0.003610,0.009778,0.003094,-0.006933,0.004517,-0.003791,-0.002358
24780,young buck wanna eat!!.. dat nigguh like I ain...,0.006171,-0.002461,-0.018769,-0.001899,-0.009897,-0.001166,-0.000997,0.009243,-0.004765,...,0.022077,-0.010136,0.003253,0.004180,-0.009084,0.002988,0.000453,-0.005204,0.001637,0.012166
24781,youu got wild bitches tellin you lies,-0.032547,-0.003426,-0.006520,0.006167,0.011983,-0.023309,0.006597,0.013260,-0.012604,...,0.008047,-0.002982,-0.004045,0.015829,0.015913,-0.007844,0.015752,-0.005909,-0.016210,-0.011625


In [133]:
removed_comments

,tweet


In [123]:
df = pd.read_csv('labeled_data.csv')

In [124]:
df

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
...,...,...,...,...,...,...,...
24778,25291,3,0,2,1,1,you's a muthaf***in lie &#8220;@LifeAsKing: @2...
24779,25292,3,0,1,2,2,"you've gone and broke the wrong heart baby, an..."
24780,25294,3,0,3,0,1,young buck wanna eat!!.. dat nigguh like I ain...
24781,25295,6,0,6,0,1,youu got wild bitches tellin you lies
